<a href="https://colab.research.google.com/github/aly202012/EnlighterJS/blob/master/linear_redression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
inputs=torch.from_numpy(inputs)
print(inputs)
print('------------------------------')
targets=torch.from_numpy(targets)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
------------------------------
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [5]:
w=torch.rand(2,3 , requires_grad=True)
b=torch.rand(2 , requires_grad=True)
print(w)
print(b)

tensor([[0.3371, 0.8819, 0.1685],
        [0.6916, 0.3311, 0.7015]], requires_grad=True)
tensor([0.8696, 0.2915], requires_grad=True)


In [6]:
w=torch.randn(2,3 , requires_grad=True)
b=torch.randn(1,2 , requires_grad=True)
print(w)
print(b)

tensor([[-0.2068, -1.3100, -0.7053],
        [-0.7497, -0.8396, -0.8350]], requires_grad=True)
tensor([[-0.0891, -0.2117]], requires_grad=True)


In [7]:
def model(x):
  return x @ w.t() + b

In [8]:
# Generate predictions
# نشاء التنبؤات 
pred=model(inputs)
print(pred)

tensor([[-133.2818, -147.1038],
        [-179.3254, -195.7670],
        [-234.5247, -226.3805],
        [-103.6074, -143.6847],
        [-189.4879, -191.0003]], grad_fn=<AddBackward0>)


In [9]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [10]:
# MSE loss

def mse(t1,t2):
  diff=t1-t2
  return torch.sum(diff*diff)/diff.numel()

In [11]:
# Compute loss
loss=mse(pred , targets)
loss

tensor(72300.9219, grad_fn=<DivBackward0>)

In [12]:
loss.backward()

In [13]:
# Gradients for weights
# تدريج الالوان 
print(w)
print(w.grad)

tensor([[-0.2068, -1.3100, -0.7053],
        [-0.7497, -0.8396, -0.8350]], requires_grad=True)
tensor([[-20251.4902, -23288.5547, -14085.2002],
        [-22788.0684, -25269.5840, -15511.5957]])


In [14]:
w
print(w.grad)

tensor([[-20251.4902, -23288.5547, -14085.2002],
        [-22788.0684, -25269.5840, -15511.5957]])


In [15]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [16]:
# Let's verify that the loss is actually lower
# دعنا نتحقق اولا من ان الخسار هي اقل بالفعل 
loss=mse(pred , targets)
print(loss)

tensor(72300.9219, grad_fn=<DivBackward0>)


In [17]:
# الان سنعمل علي تحويل التدرجات الي القيمه الصفريه وذلم استعدادا لاعاده العمليه مره اخري 

w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0.]])


In [18]:
# الان سنبدا في تنفيذ العمليات بالترتيب .
# الخطوه الاولي 
# حساب التوقعات / التنبؤات 
preds=model(inputs)
print(preds)

tensor([[ -96.8358, -106.8652],
        [-131.3856, -142.8625],
        [-177.5273, -163.6942],
        [ -67.7228, -103.8329],
        [-143.2952, -140.1569]], grad_fn=<AddBackward0>)


In [19]:
# الخطوه الثانيه 
# حساب معدل الجوده او حساب متوسط الخطا التربيعي 
loss=mse(targets , preds)
print(loss)

tensor(49088.0273, grad_fn=<DivBackward0>)


In [20]:
# الخطوه الثالثه 
# حساب التدرجات للاوزان والتحيزات 
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-16485.6172, -19233.4062, -11584.7236],
        [-18632.3672, -20800.3555, -12754.4941]])
tensor([[-199.5534, -223.4824]])


In [21]:
# Let's update the weights and biases using the gradients computed above.
#  الان ساقوم بتحديث الاوزان 

with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

In [22]:
print(w)
print('-------------------')
print(b)

tensor([[ 0.1606, -0.8847, -0.4486],
        [-0.3355, -0.3789, -0.5524]], requires_grad=True)
-------------------
tensor([[-0.0847, -0.2068]], requires_grad=True)


In [24]:
# With the new weights and biases, the model should have a lower loss.
# مع الاوزان والتحيزات الجديده لابد من ان يكون قيمه الخطا اقل 
# لذا من اجل التاكد عينا حساب التوقعات وبعدها حساب عدل / معادله الخطا 

prediction=model(inputs)
print(prediction)
print('--------------------------')
loss=mse(targets , prediction)
print(loss)

tensor([[ -66.9315,  -73.8407],
        [ -92.0421,  -99.4377],
        [-130.6910, -112.2117],
        [ -38.3488,  -71.1624],
        [-105.3448,  -98.4018]], grad_fn=<AddBackward0>)
--------------------------
tensor(33440.7617, grad_fn=<DivBackward0>)


## Train for multiple epochs


In [26]:
# هذا اللوب بعمل علي تنفيذ العمليات السابقه ما يصل الي 100 مره وفي كل مره يعمل علي حساب معادله الخطا 
for i in range(100):
  prediction=model(inputs)
  loss=mse(targets , prediction)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [27]:
prediction=model(inputs)
loss=mse(targets , prediction)
print(loss)

tensor(75.4769, grad_fn=<DivBackward0>)


In [28]:
prediction

tensor([[ 59.9936,  72.1723],
        [ 81.1275,  96.6057],
        [116.6004, 139.2063],
        [ 37.3000,  47.5023],
        [ 90.5619, 105.9078]], grad_fn=<AddBackward0>)

In [29]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [30]:
import torch.nn as nn

In [31]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [32]:
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)

In [33]:
print(inputs)
print('---------------------------------------')
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])
---------------------------------------
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


## Dataset and DataLoader


In [34]:
from torch.utils.data import TensorDataset

In [37]:
train_data=TensorDataset(inputs , targets)
train_data[:]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.],
         [ 74.,  66.,  43.],
         [ 91.,  87.,  65.],
         [ 88., 134.,  59.],
         [101.,  44.,  37.],
         [ 68.,  96.,  71.],
         [ 73.,  66.,  44.],
         [ 92.,  87.,  64.],
         [ 87., 135.,  57.],
         [103.,  43.,  36.],
         [ 68.,  97.,  70.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.],
         [ 57.,  69.],
         [ 80., 102.],
         [118., 132.],
         [ 21.,  38.],
         [104., 118.],
         [ 57.,  69.],
         [ 82., 100.],
         [118., 134.],
         [ 20.,  38.],
         [102., 120.]]))

In [47]:
# الان نحن بحاجه الي تقسيم تلك البيانات التي لدينا 
# من اجل تنفيذ ذلك سنفوم بعمل التالي 

from torch.utils.data import DataLoader

# تعريف اللودر / مقسم البيانات 
# Define data loader
batch_size = 5
train_dl = DataLoader(train_data, batch_size, shuffle=True)

In [48]:
for x_data, y_data in train_dl:
  print(x_data)
  print(y_data)
  break

tensor([[ 68.,  96.,  71.],
        [102.,  43.,  37.],
        [ 87., 135.,  57.],
        [ 91.,  87.,  65.],
        [ 69.,  96.,  70.]])
tensor([[104., 118.],
        [ 22.,  37.],
        [118., 134.],
        [ 80., 102.],
        [103., 119.]])


In [51]:
model=nn.Linear(3 , 2)
print(model.weight)
print(model.bias)
# تلك التعلميه تعرض كل الاووزان والتحيزات في الموديل الخاص بنا 

Parameter containing:
tensor([[ 0.0185,  0.5534,  0.5401],
        [ 0.4592,  0.3851, -0.0052]], requires_grad=True)
Parameter containing:
tensor([-0.1692, -0.4032], requires_grad=True)


In [52]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0185,  0.5534,  0.5401],
         [ 0.4592,  0.3851, -0.0052]], requires_grad=True),
 Parameter containing:
 tensor([-0.1692, -0.4032], requires_grad=True)]

In [53]:
# الان ساقوم بعمل الموديل / التنبؤ الفعلي بالقيم 

prediction=model(inputs)
prediction

tensor([[ 61.4820,  58.6944],
        [ 84.7779,  74.9369],
        [106.9184,  90.8444],
        [ 45.4973,  62.8018],
        [ 92.0381,  67.8831],
        [ 60.9471,  58.7686],
        [ 84.7646,  74.5466],
        [107.4770,  91.2984],
        [ 46.0322,  62.7277],
        [ 92.5597,  67.4186],
        [ 61.4687,  58.3041],
        [ 84.2430,  75.0111],
        [106.9317,  91.2347],
        [ 44.9757,  63.2663],
        [ 92.5730,  67.8089]], grad_fn=<AddmmBackward>)

## Loss Function


In [56]:
# سنقوم باستدعاء داله السار المصمنه في المكتبه والجاهزه مباشره 
# للقيام بذلك سنقوم باجراء التالي :
import torch.nn.functional as F

# Define loss function
# قمنا بتحديد نوع معادله الخطا الخاص بنا 
loss_fn = F.mse_loss
# محاوله يائسه لطباعتها
print(loss_fn)

<function mse_loss at 0x7f2c088e80e0>


In [58]:
# الان سنعمل علي تطبيق تلك المعادله علي الموديل الخاص بنا 
# نلاحظ هنا اننا قمنا باعطاء المعادله جزئين الاو وهو تنبؤ الموديل الخاص بنا والثاني هو النتائج التي من المفترض ان نحصل عليها
loss=loss_fn(model(inputs) , targets)
# وفي النتائج سنحصل علي رقم واحد والذي هو من المفترض ان يكشف عن جوده الموديل الخاص بنا 
print(loss)

tensor(670.2352, grad_fn=<MseLossBackward>)


## Optimizer

In [59]:
opt=torch.optim.SGD(model.parameters(), lr=1e-5)

In [60]:
opt

SGD (
Parameter Group 0
    dampening: 0
    lr: 1e-05
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [61]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [62]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 396.4239
Epoch [20/100], Loss: 73.9321
Epoch [30/100], Loss: 200.3985
Epoch [40/100], Loss: 136.3679
Epoch [50/100], Loss: 79.9463
Epoch [60/100], Loss: 70.1204
Epoch [70/100], Loss: 66.1902
Epoch [80/100], Loss: 34.3103
Epoch [90/100], Loss: 44.4963
Epoch [100/100], Loss: 27.1435


In [63]:
# الان لنبدا بالتنبؤ للوديل الخاص بنا 
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.5688,  72.0325],
        [ 81.8194,  97.4662],
        [117.6077, 138.5601],
        [ 24.5740,  46.1234],
        [ 99.2697, 108.0902],
        [ 56.4136,  71.0462],
        [ 81.5968,  96.8621],
        [117.8872, 138.8322],
        [ 25.7292,  47.1097],
        [100.2023, 108.4724],
        [ 57.3462,  71.4284],
        [ 80.6641,  96.4799],
        [117.8303, 139.1643],
        [ 23.6413,  45.7413],
        [100.4249, 109.0765]], grad_fn=<AddmmBackward>)

In [64]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


In [65]:
# هنا نقوم بادخال ارقام لتنبؤ 
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.2071, 68.5245]], grad_fn=<AddmmBackward>)